In [1]:
import pandas as pd
import json
from typing import Dict, List
import openai
from dotenv import load_dotenv
import os
import sys
sys.path.append('d:/perso/personal/Desktop Files/LLM projects/Elevate Repo/PowerBI-LLM')
from src.file_operator.file_operations import *
from src.json_operator.json_extraction import *
from src.json_operator.json_update import *

In [2]:
# Load environment variables
load_dotenv()
# Get API key from environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# zip_file = '../pbip examples/contenus_ftv.zip'
zip_file = '../pbip examples/inchanel global.zip'
report_json_content, model_bim_content, inner_folder_path, report_json_path, model_bim_path = extract_report_and_model(zip_file)

In [10]:
report_json_content['config']


'{"version":"5.55","themeCollection":{"baseTheme":{"name":"CY24SU06","version":"5.55","type":2}},"activeSectionIndex":0,"bookmarks":[{"displayName":"Client - since start","name":"dc960004eaa12e16bce9","explorationState":{"version":"1.3","activeSection":"cb370df3b6715abb417c","sections":{"cb370df3b6715abb417c":{"filters":{"byExpr":[{"name":"7e5e27d15b53887a9d68","type":"Categorical","filter":{"Version":2,"From":[{"Name":"d","Entity":"dim_client","Type":0}],"Where":[{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"is_chanelemployee"}}],"Values":[[{"Literal":{"Value":"false"}}]]}}}]},"expression":{"Column":{"Expression":{"SourceRef":{"Entity":"dim_client"}},"Property":"is_chanelemployee"}},"howCreated":1}]},"visualContainers":{"40ea1062e85631202166":{"filters":{"byExpr":[{"name":"Filter922f698562c398180844","type":"Categorical","filter":{"Version":2,"From":[{"Name":"d","Entity":"Date","Type":0}],"Where":[{"Condition":{"In":{"Expressions":

In [6]:
df = build_df(report_json_content)
df

,page name,visual name,visual name key,visual type,header present,title present
0,Scope FA,'Select a Boutique',title,slicer,False,True
1,Scope FA,'Select a Period',title,slicer,False,True
2,Scope FA,'FILTER mirror',title,slicer,False,True
3,Scope FA,'Select a Boutique',title,slicer,False,True
4,Scope FA,'FILTER onboarding & digital',title,slicer,False,True
5,Scope FA,'Select a Country',title,slicer,False,True
6,Scope FA,'Select a Country',title,slicer,False,True
7,Digital,'Select a Period',title,slicer,False,True
8,Digital,'Select a Country',title,slicer,False,True
9,Digital,'Select a Boutique',title,slicer,False,True


In [15]:
# user_prompt = (
#     "I want to update the slicers 'Campagne', 'Offre', 'Typologie de vidéos', 'Plateforme' and 'Catégorie' in the dashboard so that their format match the 'Catégorie' slicer on the 'Vision hebdo' page."
# )
user_prompt = "I want to make all the slicers in the Digital page and all the slicers in the FA Scope Page look like the Country slicer in the Client page"
# user_prompt = "I want to update all the slicers in the dashboard so that their format match the 'Date' slicer on the 'Client Focus' page."
# user_prompt = "I want to uniform the format of all the Country slicers in the Dashboard with the same format as the Date slicer on the Scope FA page"

In [16]:
class Agent:
    """Base class for all agents"""
    def __init__(self, name: str):
        self.name = name
        
    def log_message(self, message: str):
        print(f"[{self.name}]: {message}")
class RequestParserAgent(Agent):
    def parse_request(self, user_prompt: str, df: pd.DataFrame) -> Dict:
        # Create a prompt for the LLM to extract key information
        analysis_prompt = f"""
        Given this user request: "{user_prompt}"
        And these available pages and visuals in the dashboard:
        {df[['page name', 'visual name']].drop_duplicates().to_string()}

        Step 1: Determine the scope
        A. Page Scope Analysis:
           - If request mentions "in the dashboard" or similar → ALL pages in dashboard (INCLUDING the source page)
           - If request mentions specific pages → ONLY those pages
        
        B. Visual Scope Analysis (for each mentioned page):
           - If request EXPLICITLY mentions "all slicers" or "all visuals" → include ALL visuals in that page
           - If request mentions specific visuals → include ONLY those specific visuals
           - For dashboard-wide updates: include ALL visuals from ALL pages (including source page)

        Step 2: Identify source
        - Extract exactly one source visual and its page that will be used as reference

        Return only a JSON object with:
        {{
            "source": {{
                "page": "exact page name",
                "visual": "exact visual name"
            }},
            "page_scope": "dashboard/specific_pages",
            "targets": [
                {{
                    "page": "exact page name", // if page_scope is dashboard, MUST include all the pages
                    "include_all_visuals": true/false,
                    "specific_visuals": ["visual1", "visual2"]  // only if include_all_visuals is false
                }}
            ]
        }}

        Examples:
        1. "all slicers in the dashboard" → 
           - page_scope: "dashboard"
           - targets: ALL pages (including source page) with include_all_visuals: true
           - Note: Source page should also be included with all its visuals

        2. "Date slicer in the dashboard" → 
           - page_scope: "dashboard"
           - targets: MUST include ALL pages with specific_visuals: ["Date"]
           - Like this:
             "targets": [
                {{"page": "Page1", "include_all_visuals": false, "specific_visuals": ["Date"]}},
                {{"page": "Page2", "include_all_visuals": false, "specific_visuals": ["Date"]}},
                // ... one entry for EACH page in the dashboard
             ]

        3. "all slicers in Digital page" → 
           - page_scope: "specific_pages"
           - targets: only Digital page with include_all_visuals: true
        """
        
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": analysis_prompt}],
            temperature=0.1
        )
        
        try:
            extracted_info = json.loads(completion.choices[0].message.content)
            self.log_message(f"Extracted information: {extracted_info}")
            
            # Verify source visual exists
            source_exists = df[
                (df['page name'] == extracted_info['source']['page']) & 
                (df['visual name'] == extracted_info['source']['visual'])
            ].shape[0] > 0
            
            if not source_exists:
                self.log_message(f"Warning: Source visual '{extracted_info['source']['visual']}' not found in '{extracted_info['source']['page']}'")
                return None
            
            # Get target visuals
            targets = []
            
            # Process each target based on whether it's all visuals or specific visuals
            for target in extracted_info['targets']:
                if target['include_all_visuals']:
                    # Get all visuals for this page
                    page_visuals = df[df['page name'] == target['page']]['visual name'].unique()
                    for visual in page_visuals:
                        if not (target['page'] == extracted_info['source']['page'] and 
                               visual == extracted_info['source']['visual']):
                            targets.append({
                                "target_page": target['page'],
                                "target_visual": visual
                            })
                else:
                    # Only include the specific visuals mentioned
                    for visual in target['specific_visuals']:
                        if not (target['page'] == extracted_info['source']['page'] and 
                               visual == extracted_info['source']['visual']):
                            targets.append({
                                "target_page": target['page'],
                                "target_visual": visual
                            })
            
            return {
                "source": {
                    "source_page": extracted_info['source']['page'],
                    "source_visual": extracted_info['source']['visual']
                },
                "target": targets
            }
            
        except Exception as e:
            self.log_message(f"Error processing request: {str(e)}")
            return None
        
class EnhancedCoordinatorAgent(Agent):
    """Coordinator that manages the interaction between Parser and Critic"""
    
    def __init__(self):
        super().__init__("Coordinator")
        self.parser = RequestParserAgent("Parser")
        
    def process_request(self, user_prompt: str, df: pd.DataFrame) -> Dict:
        return self.parser.parse_request(user_prompt, df)

def process_dashboard_request(user_prompt: str, df: pd.DataFrame) -> Dict:
    coordinator = EnhancedCoordinatorAgent()
    result = coordinator.process_request(user_prompt, df)

    return result
# Test the function
result = process_dashboard_request(user_prompt, df)
print("\nFinal Configuration:")
print(json.dumps(result, indent=2))

[Parser]: Extracted information: {'source': {'page': 'Client Focus', 'visual': 'Select a Country'}, 'page_scope': 'specific_pages', 'targets': [{'page': 'Digital', 'include_all_visuals': True}, {'page': 'Scope FA', 'include_all_visuals': True}]}
[Parser]: Warning: Source visual 'Select a Country' not found in 'Client Focus'

Final Configuration:
null
